In [6]:
import napari
import numpy as np
import tifffile
import math
from typing import List
from magicgui import magicgui
from skimage.color import rgb2hsv, hsv2rgb
from napari.types import LayerDataTuple

In [7]:
CHANNEL_RANGE = None # decide if the channel range should be by hand or detected or percentil ?

In [11]:
fname = "../example_data/chroms_data_sample.tif"

## Load Image & preprocess to flat N*Channel

In [12]:
# get image
bb = tifffile.imread(fname)
# all the next step assume that the image is CZYX

bb.shape# preprocessing
# move C first
bb = np.moveaxis(bb, 1, 0)
# Flatten all dimension except C in a shape N*C
bb_flat = bb.reshape((bb.shape[0], -1)).T

# get mins and maxs for each channel to compute histogram ranges
if CHANNEL_RANGE: # defined by user
    ranges = CHANNEL_RANGE
else: # to compute automatically
    ranges = np.array([np.amin(bb_flat, axis=0), np.amax(bb_flat, axis=0)]).T


bb_flat.shape, ranges.shape

((1285120, 3), (3, 2))

In [13]:


BINS = 100 # number of bins for density histogram
THRESHOLD_VALUE = 0.4 # to remove the small value in hsv

from skimage.color import rgb2hsv

@magicgui(
    auto_call=True,
    threshold={"widget_type":"FloatSlider", "min":0, "max":1}
    )
def HS_proj(
    threshold:int,
) -> List[LayerDataTuple]:
    hsv_flat = rgb2hsv(bb_flat)
    hsv_flat = hsv_flat[hsv_flat[:,2]>threshold]
    hs_flat = hsv_flat[:,:-1] # remove Value as it shouldn't matter in cluster

    bins = BINS

    # processing
    H, edges = np.histogramdd(
        hs_flat,
        bins = bins,
        range = ((0, 1), (0, 1)), 
        density = False
    )

    size = bins
    img = np.zeros((size, size)).astype("int")
    radius = size/2.0
    cx, cy = size/2, size/2

    for x in range(size):
        for y in range(size):
            rx = x - cx
            ry = y - cy
            s = (rx ** 2.0 + ry ** 2.0) ** 0.5 / radius
            h = ((math.atan2(ry, rx) / math.pi) + 1.0) / 2.0
            if s <= 1:
                h = int(h*size-1)
                s = int(s*size-1)
                img[x, y] = H[h, s]

    # use np.log to help for the visualization
    return [
        (np.log(H), {"name":"density_HS", "colormap":"inferno"}, "image"),
        (np.log(img), {"name":"density_wheel", "colormap":"inferno"}, "image"),
        ]


size = BINS

wheel = np.zeros((size, size, 3)).astype("int")
radius = size/2.0
cx, cy = size/2, size/2

for x in range(size):
    for y in range(size):
        rx = x - cx
        ry = y - cy
        s = (rx ** 2.0 + ry ** 2.0) ** 0.5 / radius
        h = ((math.atan2(ry, rx) / math.pi) + 1.0) / 2.0
        if s <= 1:
            rgb = hsv2rgb([h, s, 1]) * 255
            wheel[x, y] = rgb   

viewer = napari.Viewer()
viewer.add_image(wheel, rgb=True)

viewer.window.add_dock_widget(HS_proj)

/tmp/ipykernel_33786/3057568602.py:45: RuntimeWarning: divide by zero encountered in log
  (np.log(H), {"name":"density_HS", "colormap":"inferno"}, "image"),
/tmp/ipykernel_33786/3057568602.py:46: RuntimeWarning: divide by zero encountered in log
  (np.log(img), {"name":"density_wheel", "colormap":"inferno"}, "image"),
/tmp/ipykernel_33786/3057568602.py:45: RuntimeWarning: divide by zero encountered in log
  (np.log(H), {"name":"density_HS", "colormap":"inferno"}, "image"),
/tmp/ipykernel_33786/3057568602.py:46: RuntimeWarning: divide by zero encountered in log
  (np.log(img), {"name":"density_wheel", "colormap":"inferno"}, "image"),
/tmp/ipykernel_33786/3057568602.py:45: RuntimeWarning: divide by zero encountered in log
  (np.log(H), {"name":"density_HS", "colormap":"inferno"}, "image"),
/tmp/ipykernel_33786/3057568602.py:46: RuntimeWarning: divide by zero encountered in log
  (np.log(img), {"name":"density_wheel", "colormap":"inferno"}, "image"),
/tmp/ipykernel_33786/3057568602.py:45